In [15]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from Custom_dataset import CustomDataset

In [16]:
adult_train = pd.read_csv('./dataset/ECG_adult_age_train.csv')
child_train = pd.read_csv('./dataset/ECG_child_age_train.csv')

In [17]:
adult_file_path = './dataset/ECG_adult_numpy_train/'
child_file_path = './dataset/ECG_child_numpy_train/'

In [18]:
adult_train_data, adult_valid_data = train_test_split(adult_train,shuffle=True, random_state=42, test_size=0.1)
adult_train_data.reset_index(inplace=True,drop=True)
adult_valid_data.reset_index(inplace=True,drop=True)

In [19]:
adult_train_dataset = CustomDataset(adult_file_path,adult_train_data)
adult_valid_dataset = CustomDataset(adult_file_path,adult_valid_data)
child_dataset = CustomDataset(child_file_path,child_train)

In [20]:
train_adult_loader = torch.utils.data.DataLoader(adult_train_dataset, batch_size = 64)
valid_adult_loader = torch.utils.data.DataLoader(adult_valid_dataset, batch_size = 64)

In [21]:
import torch
import torch.nn as nn

class Temporalblock(torch.nn.Module):
    def __init__(self, N, K, MP_factor):
        super().__init__()

        self.conv_layers = torch.nn.LazyConv1d(N, K, stride=1, padding = 'same')
        self.batchnorm_layers = torch.nn.BatchNorm1d(N)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=MP_factor, stride=MP_factor)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.batchnorm_layers(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x

class FCblock(torch.nn.Module):
    def __init__(self, N, dropout):
        super().__init__()
        self.linear = torch.nn.LazyLinear(N)
        self.batchnorm_layers = torch.nn.BatchNorm1d(N)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.linear(x)
        x = self.batchnorm_layers(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x

class spatial_block(torch.nn.Module):
    def __init__(self,N, K, MP_factor):
        super().__init__()
        self.conv_layers = torch.nn.LazyConv2d(N, (K,1),padding='same')
        self.batchnorm_layer = torch.nn.BatchNorm2d(N)
        self.relu = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool1d(kernel_size=MP_factor, stride=MP_factor)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.batchnorm_layer(x)
        x = self.relu(x)
        x = self.maxpool(x)

        return x
    
class Custom_model(torch.nn.Module):
    def __init__(self,N, K, MP_factor):
        super().__init__()
        self.first_block = Temporalblock(16, 7, 2)
        self.temporal_block = self._make_layer(N, K, MP_factor)
        self.spatial_block = spatial_block(128, 12, 2)
        self.flatten = torch.nn.Flatten()
        self.fc1 = FCblock(128, 0.2)
        self.fc2 = FCblock(64, 0.2)

        self.linear_out = torch.nn.Linear(64,1)
    def _make_layer(self, N, K, MP_factor):
        layer = []
        for n, k, mp in zip(N, K, MP_factor):
            layer.append(Temporalblock(n, k, mp))
        return torch.nn.Sequential(*layer)
    
    def forward(self,x):
        x = self.first_block(x)
        x = self.temporal_block(x)
        x = self.spatial_block(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.linear_out(x)

        return x

N = (16,32,32,64,64,64,64)
K = (5,5,5,5,3,3,3)
MP_factor = (4,2,4,2,2,2,2)
model = Custom_model(N, K, MP_factor)
model.to('cuda')
print('model_ready')

model_ready


c:\Anaconda\envs\colon\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [22]:
model

Custom_model(
  (first_block): Temporalblock(
    (conv_layers): LazyConv1d(0, 16, kernel_size=(7,), stride=(1,), padding=same)
    (batchnorm_layers): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (temporal_block): Sequential(
    (0): Temporalblock(
      (conv_layers): LazyConv1d(0, 16, kernel_size=(5,), stride=(1,), padding=same)
      (batchnorm_layers): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (maxpool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Temporalblock(
      (conv_layers): LazyConv1d(0, 32, kernel_size=(5,), stride=(1,), padding=same)
      (batchnorm_layers): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (maxpool): MaxPool1d(kernel_size=2, stride=2, pad

In [23]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr = 3e-4)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', min_lr = 1e-5, patience=2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max= 20, eta_min=1e-5)

In [24]:
for epochs in range(20):
    print('train_run')
    for batch in tqdm(train_adult_loader):
        inputs = batch[0].view(-1,12,5000).to('cuda')
        label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
        output = model(torch.tensor(inputs, dtype = torch.float32))
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    valid_loss = 0
    print('valid_run')
    for batch in tqdm(valid_adult_loader):
        inputs = batch[0].view(-1,12,5000).to('cuda')
        label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
        with torch.no_grad():
            output = model(torch.tensor(inputs, dtype = torch.float32))
        loss = criterion(output, label)
        valid_loss += loss
    loss_check = valid_loss/len(valid_adult_loader)
    print(loss_check)
    scheduler.step(loss_check)

train_run


  0%|          | 0/491 [00:00<?, ?it/s]C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_9864\2323642984.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_9864\2323642984.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs, dtype = torch.float32))
c:\Anaconda\envs\colon\lib\site-packages\torch\nn\modules\conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\nati

ValueError: expected 4D input (got 3D input)

In [10]:
pred = []
labels = []
for batch in tqdm(valid_adult_loader):
    inputs = batch[0].view(-1,12,5000).to('cuda')
    label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
    with torch.no_grad():
        output = model(torch.tensor(inputs, dtype = torch.float32))
    pred += output.detach().cpu().tolist()
    labels += label.detach().cpu().tolist()

  0%|          | 0/55 [00:00<?, ?it/s]C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_1180\1463697845.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_1180\1463697845.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs, dtype = torch.float32))
100%|██████████| 55/55 [00:02<00:00, 24.48it/s]


In [14]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(pred,labels)

13.689496059193717